In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import sys
sys.path.append('..')
from green_city.utils import span
from green_city.plotting import plot_balance
from green_city.simulation import Battery
import plotly.graph_objects as go

plt.rcParams['figure.figsize'] = [15, 5]

building_nr = 5

In [ ]:
def get_building_properties(building_nr):
    with open("../data/citylearn_challenge_2021/schema.json") as schema_file:
        schema = json.load(schema_file)

    building_properties = schema['buildings'][f'Building_{building_nr}'] 
    return building_properties

In [ ]:
df = pd.read_csv(f"../data/preprocessed/Building_{building_nr}.csv").astype({'datetime': 'datetime64'}).set_index('datetime')

## Energy balance without the use of a battery

In [ ]:
#buiding 4 does not seem to have any heating installed.
if building_nr == 4:
    df["dhw_heating_kWh"].unique()

In [ ]:
timespan = span("2009-02-08", "2009-02-12")
adding = ['equipment_electric_power_kWh', 'dhw_heating_kWh', 'cooling_load_kWh']
substracting = ['solar_generation_kW']
plot_balance(timespan, adding, substracting, df)

In [ ]:
df.loc[timespan, substracting].plot.area()
df.loc[timespan, adding].plot.area()

- the equipment power usage does not look like 'natural' consumption, as this is "strip mall retail (Building_4),"

In [ ]:
df.columns

In [ ]:
#"cooling over the seasons:"
#'equipment_electric_power_kWh', 'dhw_heating_kWh', 'cooling_load_kWh'
df.loc[:, 'dhw_heating_kWh'].plot.area()
#how again would I get daily averages?

In [ ]:
import plotly.express as px
fig = px.scatter(df, x='outdoor_temp', y='dhw_heating_kWh', trendline="ols", trendline_color_override="green")
fig.show()
fig = px.scatter(df, x='outdoor_temp', y='equipment_electric_power_kWh', trendline="ols", trendline_color_override="green")
fig.show()
fig = px.scatter(df, x='outdoor_temp', y='cooling_load_kWh', trendline="ols", trendline_color_override="green")
fig.show()


In [ ]:
df.index.date

In [ ]:
df["date"] = df.index.date
df_daily = df.groupby("date").mean()
fig = px.scatter(df_daily, x='outdoor_temp', y='dhw_heating_kWh', trendline="ols", trendline_color_override="green")
fig.show()

In [ ]:
df["date"] = df.index.date
df["year"] = df.index.year
df["week"] = df.index.isocalendar().week
df['year_week'] = df.agg('{0[year]}_{0[week]:02d}'.format, axis=1)
df.year_week
df_weekly = df.groupby("year_week").mean()


In [ ]:
from sklearn.linear_model import LinearRegression
coefs = []
for period in range(2, 250):
    #df.index.map(lambda x: x.year + x.isocalendar().week)
    df_weeks = df.assign(week_total = df.index.map(lambda x: (x - pd.Timestamp(2008, 1, 2)).days//period + 1))
    df_weeks_means = df_weeks.groupby("week_total").mean()
    #fig = px.scatter(df_weeks_means, x='outdoor_temp', y='dhw_heating_kWh', trendline="ols", trendline_color_override="green")
    #fig.show()

    reg = LinearRegression().fit(df_weeks_means[["outdoor_temp"]], df_weeks_means.dhw_heating_kWh)
    print(period, ":", reg.coef_)
    coefs.append(reg.coef_)
plt.plot(coefs)

df_weeks = df.assign(week_total = df.index.map(lambda x: (x - pd.Timestamp(2008, 1, 2)).days//127 + 1))
df_weeks_means = df_weeks.groupby("week_total").mean()
fig = px.scatter(df_weeks_means, x='outdoor_temp', y='dhw_heating_kWh', trendline="ols", trendline_color_override="green")
fig.show()

### TASK: predict DHW heating from outdoor temperature and indoor temperature over a certain period.

## Using a battery to manage load

In [ ]:
#trying out the battery model from the CityLearn simulation
building_properties = get_building_properties(building_nr)
battery_params = building_properties["electrical_storage"]['attributes']
battery_params["capacity_loss_coef"] =  battery_params.pop("capacity_loss_coefficient")
battery_params["loss_coef"] =  battery_params.pop("loss_coefficient")
print(battery_params)

battery = Battery(**battery_params)

print("Charging and discharging the battery")
print(battery._soc, "initial")
battery.charge(1.0)
print(battery._soc, "initial")
battery.charge(9.3)
print(battery._soc, "after charging 9.3")
battery.charge(0.3)
print(battery._soc, "after charging 0.3")
battery.charge(-0.3)
print(battery._soc, "after discharging 0.3")